In [1]:
import numpy as np
import pandas as pd

## READ THE DATA

In [2]:
fileName = 'input.txt'
with open(fileName) as openFile:
    fileList = list(openFile)

earliestDeparture = int(fileList[0])
listOfBuses = [int(num) if num.isdigit() else num for num in fileList[1].rstrip().split(',')]
listOfBusesWithoutX = [num for num in listOfBuses if str(num).isdigit()]

## PART 1

In [3]:
increment = 0
foundBus = False

while True:
    for bus in listOfBusesWithoutX:
        if ((earliestDeparture+increment) % bus == 0):
            foundBus = True
            earliestBusAvailable = bus
            break 
    
    if foundBus:
        break
    increment += 1

print("Earliest bus:", earliestBusAvailable)
print("Minutes to wait:", increment)
print("Answer:", earliestBusAvailable*increment)

Earliest bus: 17
Minutes to wait: 9
Answer: 153


## PART 2
This is easier solved through the use of the [Chinese remainder theorem](https://en.wikipedia.org/wiki/Chinese_remainder_theorem). First we check if our bus IDs are all coprime.

In [4]:
def GreatestCommonDivisor(p,q):
    while q != 0:
        p, q = q, p%q
    return p

def IsCoprime(x, y):
    return GreatestCommonDivisor(x, y) == 1

In [5]:
coprime = True

for i in range(len(listOfBusesWithoutX)):
    for j in range(i+1,len(listOfBusesWithoutX)):
            coprime *= IsCoprime(int(listOfBusesWithoutX[i]), int(listOfBusesWithoutX[j]))
            if not coprime:
                print("ERROR: IDs", listOfBusesWithoutX[i], "and", listOfBusesWithoutX[j], "are not coprime.")

if coprime:
    print("Great! All bus IDs are coprime!")

Great! All bus IDs are coprime!


The Chinese remainder theorem tells us that a system of congruences in the form

\begin{equation}
x\, \text{mod}\, n_i = b_i
\end{equation}

has a single solution up to modulus $N := \prod_{i}n_i$. Said solution takes the form

\begin{equation}
x\, \text{mod}\, N = \sum_{i=1}^m b_i N_i x_i
\end{equation}

where $N_i := \frac{N}{n_i}$ and $x_i$ is the solution of

\begin{equation}
(N_i x_i) \, \text{mod}\, n_i = 1
\end{equation}

In our case, $x$ is the timestamp we're looking for; since every bus bar the first departs with a $k_i$ minute offset, most of our congruences will actually be in the form

\begin{equation}
(x+k_i)\, \text{mod}\, n_i = 0
\end{equation}

Thankfully, it can easily be shown that the above equation is equivalent to

\begin{equation}
x\, \text{mod}\, n_i = (n_i-k_i)\, \text{mod}\, n_i
\end{equation}

In [6]:
niArray = np.array(listOfBusesWithoutX, dtype='uint64')
N = np.prod(niArray, dtype='uint64')
NiArray = np.array([int(N/ni) for ni in niArray], dtype='uint64')

In [7]:
xiList = []
for i in range(len(niArray)):
    xi = 0
    while (NiArray[i] * xi) % niArray[i] != 1:
        xi += 1
    xiList.append(xi)

xiArray = np.array(xiList, dtype='uint64')

In [8]:
nikiDictionary = {}
for i in range(len(listOfBuses)):
    if listOfBuses[i] != 'x':
        nikiDictionary[listOfBuses[i]] = i

In [9]:
biArray = np.array([ni - nikiDictionary[ni] for ni in niArray], dtype='uint64')

In [10]:
def CheckTimestampCondition(minuteDictionary, timestamp):
    for bus in minuteDictionary:
        if (timestamp+minuteDictionary[bus]) % bus != 0:
            return False 
    return True

In [11]:
solution = (np.sum(biArray*NiArray*xiArray)) % N
print("The unique solution is:", solution)
print("Does it satisfy the requirements?", CheckTimestampCondition(nikiDictionary, solution))

The unique solution is: 471793476184394
Does it satisfy the requirements? True
